In [43]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from gensim.models import KeyedVectors

In [2]:
df = pd.read_json("/Users/ramir/Desktop/Carpeta/Nueva_carpeta/Programas/Laboratorios_IA/Lab_1/complaints.json")

In [3]:
df = df[['complaint_what_happened', 'product', 'company', 'state', 'consumer_disputed',]].dropna()
df = df[df['consumer_disputed'] != 'N/A']
df = df[df['complaint_what_happened'] != '']

In [7]:
df_temporal = pd.DataFrame()
df_temporal = df_temporal.append(df[df['consumer_disputed'] == 'No'].iloc[:35807], ignore_index=True)
df = df_temporal.append(df[df['consumer_disputed'] == 'Yes'], ignore_index=True)
df

C:\Users\ramir\AppData\Local\Temp\ipykernel_12932\1485403451.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_temporal = df_temporal.append(df[df['consumer_disputed'] == 'No'].iloc[:35807], ignore_index=True)
C:\Users\ramir\AppData\Local\Temp\ipykernel_12932\1485403451.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_temporal.append(df[df['consumer_disputed'] == 'Yes'], ignore_index=True)


,complaint_what_happened,product,company,state,consumer_disputed
0,Denied sufficient time to complete a ( short s...,Mortgage,NATIONSTAR MORTGAGE LLC,TN,No
1,Nationstar Mortgage withheld money from my mon...,Mortgage,NATIONSTAR MORTGAGE LLC,GA,No
2,My client is XXXX XXXX and I have been working...,Mortgage,NATIONSTAR MORTGAGE LLC,MD,No
3,My original note was owned by XXXX Bank and I ...,Mortgage,NATIONSTAR MORTGAGE LLC,CA,No
4,My mortgage was bought by Nationstar Mortgage ...,Mortgage,NATIONSTAR MORTGAGE LLC,NY,No
...,...,...,...,...,...
71609,"On, XXXX/XXXX/2015 I received a phone call fro...",Consumer Loan,Western-Shamrock Corporation,AL,Yes
71610,I graduated from XXXX in XX/XX/XXXX. I did not...,Student loan,DISCOVER BANK,AZ,Yes
71611,i am behind on my mortgage and was dealing wit...,Mortgage,RUSHMORE LOAN MANAGEMENT SERVICES LLC,NY,Yes
71612,I opened a checking account in XX/XX/XXXX with...,Bank account or service,"CITIBANK, N.A.",IL,Yes


In [38]:
# Preprocess the text data
texts = df['complaint_what_happened'].values
labels = df['consumer_disputed'].map({'Yes': 1, 'No': 0}).values

In [39]:
# Split into trainning and test sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [40]:
# Tokenize the data
max_words = 23300
max_len = 400

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [79]:
import pickle

# Save the Tokenizer to a file
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [44]:
from gensim.models import KeyedVectors

# Loading the pre-trained Word2Vec model (Google's Word2Vec model)
word2vec_model = KeyedVectors.load_word2vec_format('Google_word2vec.bin', binary=True)

embedding_matrix = np.zeros((max_words, word2vec_model.vector_size))

for word, i in tokenizer.word_index.items():
    if i < max_words and word in word2vec_model:
        embedding_matrix[i] = word2vec_model[word]

In [45]:
# Define the model to use
model = Sequential()
model.add(Embedding(max_words, word2vec_model.vector_size, weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [46]:
# Traine the model
model.fit(X_train_pad, y_train, batch_size=32, epochs=10, validation_data=(X_test_pad, y_test))

Epoch 1/10
1791/1791 [==============================] - 784s 436ms/step - loss: 0.6895 - accuracy: 0.5376 - val_loss: 0.6896 - val_accuracy: 0.5351
Epoch 2/10
1791/1791 [==============================] - 772s 431ms/step - loss: 0.6760 - accuracy: 0.5787 - val_loss: 0.6743 - val_accuracy: 0.5878
Epoch 3/10
1791/1791 [==============================] - 809s 452ms/step - loss: 0.6655 - accuracy: 0.5994 - val_loss: 0.6732 - val_accuracy: 0.5771
Epoch 4/10
1791/1791 [==============================] - 939s 524ms/step - loss: 0.6620 - accuracy: 0.6012 - val_loss: 0.6714 - val_accuracy: 0.5933
Epoch 5/10
1791/1791 [==============================] - 2352s 1s/step - loss: 0.6580 - accuracy: 0.6100 - val_loss: 0.6694 - val_accuracy: 0.5925
Epoch 6/10
1791/1791 [==============================] - 751s 419ms/step - loss: 0.6499 - accuracy: 0.6201 - val_loss: 0.6709 - val_accuracy: 0.5891
Epoch 7/10
1791/1791 [==============================] - 765s 427ms/step - loss: 0.6422 - accuracy: 0.6288 - val_lo

In [47]:
# Test the model accuracy
score = model.evaluate(X_test_pad, y_test)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')

448/448 [==============================] - 150s 334ms/step - loss: 0.7049 - accuracy: 0.5831
Test loss: 0.7048846483230591
Test accuracy: 0.5831180810928345


In [76]:
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)